<a href="https://colab.research.google.com/github/ha-pu/data_course/blob/AOM25/4-machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

+ title: Machine Learning
+ author: Harald Puhr
+ date: July 20, 2025

[Data Source](https://github.com/ha-pu/data_files#repurchase-likelihood-dataset)

# Load packages and data

In [ ]:
install.packages(c("caret", "caretEnsemble", "gbm", "glmnet", "randomForest", "tidyverse", "xgboost"))

In [ ]:
library(caret)
library(caretEnsemble)
library(tidyverse)

Setting a seed is important for reproducibility. It ensures that the random
processes in your code yield the same results each time you run it. This is
particularly crucial in machine learning, where randomness can significantly
affect model training and evaluation.

In [ ]:
set.seed(123)

In [ ]:
df <- read_csv("https://raw.githubusercontent.com/ha-pu/data_files/refs/heads/main/4-repurchase_likelihood.csv")

# Preparing our data

The implementations of most machine learning algorithms in R have very specific
requirements regarding the coding of input data. This means we have to take
more steps to prepare the data than when working with simple regressions.

## Inspect data frame

In [ ]:
slice_head(df)

In [ ]:
glimpse(df)

In [ ]:
summary(df)

## Recode variables

Code variables `repurchase`, `customer_sex`, and `customer_tier` as type factor
to better handle categorical data.

In [ ]:
df <- df %>% 
  mutate(
    repurchase = factor(repurchase),
    customer_sex = factor(customer_sex),
    customer_tier = factor(customer_tier)
  )

## Drop columns which do not contain relevant information

Our data set includes only one observation by customer. This means by including
the variables `customer_id` and `customer_name`, we would create
customer-specific predictions. This is not what we want to do! We want to
generalize from our existing customers to new customers.

In [ ]:
df <- select(df, -customer_id, -customer_name)

## Transform categorical predictors into dummy variables

R stores categorical data as factors (e.g., variable = A, B, A, A, B). For
machine learning, we transform them into separate columns:
(variable.A = 1, 0, 1, 1, 0; variable.B = 0, 1, 0, 0, 1).

In [ ]:
dummyModel <- dummyVars(repurchase ~ ., df, fullRank = TRUE)

df <- predict(dummyModel, newdata = df) %>%
  as.data.frame() %>%
  mutate(repurchase = df$repurchase)

## Designate he column names of our outcome and predictor variables

In [ ]:
label <- "repurchase"
predictors <- names(df)[!(names(df) %in% label)]

# Splitting the data into training and test sets

If we test the model on the same data as we train it, we can only verify how
well the algorithm fits the data it has seen during training. This is not a good
measure of accuracy. We want to know how well it fits data it has **not** seen
in training. Therefore, we put part of the data aside for testing.

In [ ]:
train_idx <- sample(seq(nrow(df)), size = nrow(df) * 0.8)
train_df <- slice(df, train_idx)
test_df <- slice(df, -train_idx)

# Pre-processing the data

## Change data type of the label variable

In [ ]:
y_train <- train_df[,label]
y_test <- test_df[,label]

# Ensure levels of y_train are valid R variable names
levels(y_train) <- make.names(levels(y_train))
levels(y_test) <- make.names(levels(y_test))

head(y_train)

## Avoid multicollinearity

In [ ]:
findCorrelation(cor(train_df[,predictors]))

## Check for missing values and use median imputation

In [ ]:
summary(train_df)

Machine learning algorithms do not handle missing data well. We, therefore,
impute missing data with the sample median.

In [ ]:
preproc_par <- preProcess(train_df[,predictors], method = c("medianImpute"))
train_df[,predictors] <- predict(preproc_par, train_df[,predictors])

## Scale and center the data

Machine learning algorithms do not handle differences in the scaling of
variables well. This is an issue if we use variables that have different data
ranges in the same model (e.g., Total GDP and GPD/capita). To solve the issue,
we standardize numeric predictors to *mean = 0* and *SD = 1*.

In [ ]:
preproc_par <- preProcess(train_df[,predictors], method=c("center", "scale"))
train_df[,predictors] <- predict(preproc_par, train_df[,predictors])

In [ ]:
preprocPar <- preProcess(test_df[,predictors], method=c("center", "scale", "medianImpute"))
test_df[,predictors] <- predict(preprocPar, test_df[,predictors])

# Model training, validation, and testing

## Algorithms

In this exercise, we will start with five frequently-used machine learning
algorithms:

* Random Forest (`method = "rf"`):
  * What it does: Random Forest is an ensemble learning method that builds
    multiple decision trees and merges their outputs for more accurate and
    stable predictions.
  * How it works: Each tree is trained on a random sample (with replacement) of
    the data, and at each split in the tree, a random subset of features is
    considered. The final prediction is made by majority vote (classification)
    or average (regression).
* Generalized Linear Model with Elastic Net Regularization (`method = "glmnet"`):
  * What it does: This model combines ridge and lasso regression, useful for
    handling multicollinearity and for variable selection.
  * How it works: It minimizes the residual sum of squares with both L1 (lasso)
    and L2 (ridge) penalties. The balance between these penalties is controlled
    by a mixing parameter (`alpha`).
* Neural Network (`method = "nnet"`):
  * What it does: A simple feedforward neural network for classification or
    regression.
  * How it works: It uses one hidden layer with a customizable number of
    neurons. The network learns weights for input features through
    backpropagation and gradient descent, optimizing to reduce prediction error.
* XGBoost with Decision Trees (`method = "xgbTree"`):
  * What it does: XGBoost is an efficient implementation of gradient boosting
    using decision trees.
  * How it works: It builds trees sequentially, where each new tree corrects the
    errors of the previous ones. It uses gradient descent to minimize a loss
    function and includes regularization to prevent overfitting.
* XGBoost with Linear Base Learners (`method = "xgbLinear"`):
  * What it does: Like `xgbTree`, but uses linear models as base learners
    instead of trees.
  * How it works: It applies gradient boosting to linear models (like linear
    regression), learning coefficients that minimize the loss function, with
    regularization terms to avoid overfitting.

## Training

We use 5-fold cross-validation (CV) for training. CV means that the model is
train on 4/5 of the data and validates it on the remaining 1/5 of the data. This
process is repeated 4 times (once on every fold). Then, we take the average of
these 5 evaluations to get the final evaluation of the respective training step.

In [ ]:
quickTrain <- trainControl(method = "cv", number = 5)

model_rf <- train(x = train_df[,predictors], y = y_train, method = "rf", trControl = quickTrain)

model_glmnet <- train(x = train_df[,predictors], y = y_train, method = "glmnet", trControl = quickTrain)

model_nnet <- train(x = train_df[,predictors], y = y_train, method = "nnet", trControl = quickTrain)

model_xgbTree <- train(x = train_df[,predictors], y = y_train, method = "xgbTree", trControl = quickTrain)

model_xgbLinear <- train(x = train_df[,predictors], y = y_train, method = "xgbLinear", trControl = quickTrain)

### Training process

In [ ]:
model_glmnet %>%
  ggplot(highlight = TRUE) +
  theme_bw()

In [ ]:
model_nnet %>%
  ggplot(highlight = TRUE) +
  theme_bw()

## Validation

In [ ]:
comparison_alg <- data.frame(
  Model = c("rf","glmnet","nnet","xgbTree","xgbLinear"),
  Accuracy_base = c(
    max(model_rf$results$Accuracy), max(model_glmnet$results$Accuracy),
    max(model_nnet$results$Accuracy), max(model_xgbTree$results$Accuracy),
    max(model_xgbLinear$results$Accuracy))
)

comparison_alg %>%
  ggplot() +
  geom_bar(aes(x = Model, y = Accuracy_base), stat = "identity") +
  geom_hline(aes(yintercept = max(Accuracy_base)), linetype = "dashed", color = "darkred") +
  scale_y_continuous(limits = c(0.0, 0.9)) +
  theme_bw()

comparison_alg

### Additional validation methods

We repeat the analysis with 10-fold cross-validation and bootstrapping. In
bootstrapping, we create sub-samples of the data by sampling with replacement.
This means that some observations may be included multiple times in a
sub-sample, while others may not be included at all. The model is trained on
these sub-samples, and the process is repeated multiple times to obtain a more
robust estimate of the model's performance.

In [ ]:
# Cross validation training method
crossValidation <- trainControl(method = "cv", number = 10)

# Bootstrapping training method
bootstraping <- trainControl(method = "boot")

# Apply both methods to a random forest model
model_cv <- train(x = train_df[,predictors], y = y_train, method = "rf", trControl = crossValidation)
model_boot <- train(x = train_df[,predictors], y = y_train, method = "rf", trControl = bootstraping)

In [ ]:
comparison_val <- data.frame(
  Model = c("cv5", "cv10","boot"),
  Accuracy_base = c(
    max(model_rf$results$Accuracy), max(model_cv$results$Accuracy),
    max(model_boot$results$Accuracy)
  )
)

comparison_val %>%
  ggplot() +
  geom_bar(aes(x = Model, y = Accuracy_base), stat = "identity") +
  geom_hline(aes(yintercept = max(Accuracy_base)), linetype = "dashed", color = "darkred") +
  scale_y_continuous(limits = c(0.0, 0.9)) +
  theme_bw()

comparison_val

### Model tuning

Through tuning, we can optimize the hyper-parameters of our model to improve its
learning behavior. Hyper-parameters are parameters that are set before the
learning process begins. They are not learned from the data but are instead
specified by the user before training the model. Examples include
the learning rate, the number of trees in a random forest, the number of hidden
layers in a neural network, and the regularization strength in a linear model.

We test three approaches to tuning:

* Grid search: Systematically evaluates a predefined grid of hyper-parameter
  combinations.
* Random search: Randomly samples parameter combinations from the parameter
  space.
* Adaptive search: Starts with a random search (or grid), but learns from
  performance during tuning and stops testing unpromising combinations early.

In [ ]:
# Grid tuning
gridTune <- trainControl(method = "cv", search = "grid")

# Random search tuning method:
randomTune <- trainControl(method = "cv", search = "random")

# Adaptive search tuning method:
adaptiveTune <- trainControl(
  method = "adaptive_cv",
  adaptive = list(min = 2, alpha = 0.05, method = "gls", complete = TRUE),
  search = "random"
  )

# Apply all three methods to a glmnet model
model_grid <- train(
  x = train_df[,predictors], y = y_train, method = "glmnet",
  trControl = gridTune, tuneLength = 5
  )
model_random <- train(
  x = train_df[,predictors], y = y_train, method = "glmnet",
  trControl = randomTune, tuneLength = 25
  )
model_adaptive <- train(
  x = train_df[,predictors], y = y_train, method = "glmnet",
  trControl = adaptiveTune, tuneLength = 25
  )

In [ ]:
comparison_tune <- data.frame(
  Model = c("cv5", "grid", "random", "adaptive"),
  Accuracy_base = c(
    max(model_glmnet$results$Accuracy), max(model_grid$results$Accuracy),
    max(model_random$results$Accuracy), max(model_adaptive$results$Accuracy)
  )
)

comparison_tune %>%
  ggplot() +
  geom_bar(aes(x = Model, y = Accuracy_base), stat = "identity") +
  geom_hline(aes(yintercept = max(Accuracy_base)), linetype = "dashed", color = "darkred") +
  scale_y_continuous(limits = c(0.0, 0.9)) +
  theme_bw()

comparison_tune

## Testing

In [ ]:
comparison_all <- bind_rows(comparison_alg, comparison_val, comparison_tune) %>%
  filter(Model != "cv5")

det_OOS_accuracy <- function(model){
  confusionMatrix(predict(model, test_df), y_test)$overall[[1]]
}

all_model_list <- list(
  model_rf, model_glmnet, model_nnet, model_xgbTree,
  model_xgbLinear, model_cv, model_boot, model_grid,
  model_random, model_adaptive
)

comparison_all <- mutate(
  comparison_all,
  OOS_Acc = map_dbl(all_model_list, det_OOS_accuracy)
)

comparison_all %>%
  pivot_longer(
    cols = c(Accuracy_base, OOS_Acc),
    names_to = "Base",
    values_to = "Accuracy"
  ) %>%
  ggplot() +
  geom_line(aes(x = Model, y = Accuracy, color = Base, group = Base)) +
  geom_point(aes(x = Model, y = Accuracy, color = Base)) +
  geom_hline(aes(yintercept = max(comparison_all$Accuracy_base)), colour = "darkred", linetype = "dashed") +
  geom_hline(aes(yintercept = max(comparison_all$OOS_Acc)), colour = "darkblue", linetype = "dashed") +
  scale_y_continuous(limits = c(0.0, 0.9)) +
  theme_bw()

arrange(comparison_all, desc(OOS_Acc))

### Variable importance

In [ ]:
varImp(model_xgbTree)

In [ ]:
plot(varImp(model_xgbTree), main = "xgbTree")

In [ ]:
plot(varImp(model_nnet), main = "nnet")

In [ ]:
plot(varImp(model_glmnet), main = "glmnet")

# Ensemble learning

Ensemble learning is a machine learning technique that combines predictions from
multiple models to improve accuracy, robustness, and generalization. The idea is
simple but powerful: A group of diverse models working together is usually
better than any single one.

There are mulitpe ways to combine models:
* Bagging: Build many models on different random subsets of the data and combine
  them via averaging (regression) or voting (classification).
* Boosting: Build models sequentially, where each model learns from the errors
  of the previous and combine them with weighted averaging.
* Stacking: Build several base models and train a meta-model on their predictions.

We first define a list of models and apply them to the same folds of our data.

In [ ]:
algoList <- c("rf", "glmnet", "nnet", "xgbTree", "xgbLinear")

stack_index = createFolds(y_train, k = 5) # We want to fix our index for comparison and validity

listControl <- trainControl(
  method = "cv", number = 5, 
  classProbs = TRUE, # We need to predict probabilities not 0 or 1
  savePredictions = TRUE, # necessary for stack model later
  index = stack_index
)

models <- caretList(
  x = train_df[,predictors], y = y_train, trControl = listControl,
  methodList = algoList
)

## Compare the models

In [ ]:
results <- resamples(models)
summary(results)
modelCor(resamples(models))

## Automatic model selection

Next, we use a meta model to combine the predictions of the base models. The
meta model can be a simple linear regression model or a more complex model like
an extreme gradient boosting tree (`xgbTree`), a gradient boosting machine
(`gbm`), or a simple logistic regression (`glm`). GBM is an ensemble learning
method that builds a strong predictive model by sequentially combining many weak
models, typically decision trees.

In [ ]:
stackControl <- trainControl(
  method = "repeatedcv", number = 10, repeats = 3, classProbs = TRUE,
  index = stack_index
  )

stack_tree <- caretStack(models, method = "xgbTree", trControl = stackControl)

stack_gbm <- caretStack(models, method = "gbm", trControl = stackControl)

stack_glm <- caretStack(models, method = "glm", trControl = stackControl)

## Out-of-Sample Accuracy

In [ ]:
results <- data.frame(
  ensemble_tree = predict(stack_tree, test_df)$X1,
  ensemble_glm = predict(stack_glm, test_df)$X1,
  ensemble_gbm = predict(stack_gbm, test_df)$X1
)

results <- predict(models, test_df) %>%
  as.data.frame() %>% 
  bind_cols(results)

det_OOS_accuracy_probs <- function(model){
  preds <- factor(model >= 0.5, labels = c("X0", "X1"))
  confusionMatrix(preds, y_test)$overall[[1]]
}

OOS_ACC <- results %>%
  summarise(across(everything(), det_OOS_accuracy_probs))

OOS_ACC

OOS_ACC %>%
  pivot_longer(cols = everything(), names_to = "Model", values_to = "OOS_ACC") %>%
  ggplot() +
  geom_bar(aes(x = Model, y = OOS_ACC), stat = "identity") + 
  geom_hline(aes(yintercept = max(OOS_ACC)), linetype = "dashed", color = "darkred") +
  labs(
    x = NULL
  ) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))